In [1]:
"""This is video reader with all operations inside fnctions, so they can be called on 
    each frame out of a set of frames, and each video out of a set of videos
    """

'This is video reader with all operations inside fnctions, so they can be called on \n    each frame out of a set of frames, and each video out of a set of videos\n    '

In [2]:
"""Imports"""
import matplotlib.pyplot as plt #for maknig plots inside the notebook
import skimage
import skvideo.io
from skvideo.io import vreader, ffprobe
from skimage import measure, morphology, feature
from skimage.filters import *
from skimage.morphology import *
from operator import attrgetter
import numpy as np
import math
from itertools import chain
#from functools import partial, update_wrapper
from scipy import ndimage 
#from pims import pipeline, Video
from glob import glob
import seaborn as sns
from collections import OrderedDict 
#import antigravity

In [3]:
"""Functions"""
# apply total thresholding to each of a list of frames; filter slides with wrong number

def total_threshold_filter(frame, frame_no, transition_threshold, class_thresh,
                           broken_count, class_num, origin, last_class, last_whole, state_start, filtrate_len, 
                           classes, transitions, times): 
    
   
    
   # ---setup image and detect blobs ----------------
    thresh_img = frame > threshold_isodata(frame)# binary image
        
    phighlight = morphology.binary_opening(thresh_img, disk(8))
    img = np.copy(frame)
    img[phighlight==0] = 0
    pblobs =skimage.feature.blob_doh(img, min_sigma =8, max_sigma = 17, threshold = 0.007, num_sigma= 15, overlap=0.6)
        
    ghighlight = gaussian(
        morphology.binary_opening(
             (thresh_img^phighlight)
                           , square(4)), sigma = 0.65)
    img = np.copy(frame)
    img[ghighlight==0] = 0
    gblobs =skimage.feature.blob_doh(img, min_sigma =9, max_sigma = 16, threshold = 0.005, num_sigma= 15, overlap=0.2)
    

    ngblobs = len(gblobs)
    npblobs = len(pblobs) 
    num_blobs = ngblobs + npblobs
   
    glas_on_glas_dists =np.array([connect((gblobs[i][1], gblobs[i][0]),(g_bond, fg_bond), gblobs[i+1:]) for i in range(ngblobs-1)])
    num_gconnections, num_fgconnections = reducer(glas_on_glas_dists)


    plas_on_plas_dists = np.array([connect((pblobs[i][1], pblobs[i][0]),(p_bond, fp_bond), pblobs[i+1:]) for i in range(npblobs-1)])
    num_pconnections, num_fpconnections = reducer(plas_on_plas_dists)


    num_connections = sum([num_pconnections, num_gconnections])
    
    #----------------------filter------------------------- can use actual filter
    """test = (num_connections >= expected_connections and num_blobs>=expected_blobs)
    if test: # keep frames that have enough paricles, and are not transitions"""
    clas =sideify(num_pconnections,num_gconnections, num_fpconnections, num_fgconnections )
    #clas = side
    #sides[side].append(frame_no)
    classes[clas]+=1
    #print(clas)

    if clas == 'ucf':
        broken_count += 1
        class_num = 0
        #print('ucf-broken')
        #print("class_num: %s" %class_num)
        #print("broken_count: %s" %broken_count)
        pass
    else:
        if origin == 'ucf':
            origin = last_class
            #print("reset origin", origin)
        else:
            pass
            
        if clas == last_class:
            class_num += 1
            #print("class_num: %s" %class_num)
        else:
            class_num = 0
            #print("class_reset")
            #print("class_num: %s" %class_num)
            
        if class_num >= class_thresh: 
            if broken_count >= transition_threshold or clas != origin:
                transitions[origin + "->" + clas]+= 1#((last_whole, frame_no))
                
                between_time = frame_no - state_start                               
                times[origin + "->" + clas][0].append(between_time)
                
                stable_time = last_whole - state_start
                times[origin + "->" + clas][1].append(stable_time)
                
                unstable_time = frame_no - last_whole                      
                times[origin + "->" + clas][2].append(unstable_time)
                
                
                state_start = frame_no
                
                #print(origin + "->" + clas)
                origin = clas
                #print("origin: %s" %origin)
            broken_count = 0
            #print("broken_count: %s" %broken_count)
            last_whole = frame_no
        else:
            pass


        
        filtrate_len += 1 
        last_class = clas
            
            
    """else:
        print("broken")
        class_num = 0
        print("class_num: %s" %class_num)
        broken_count += 1
        print("broken_count: %s" %broken_count)"""

    return [broken_count, class_num, origin, last_class, last_whole,state_start, filtrate_len]



        
# take largest isodata image region, and return aclassification
def sideify(num_pconnections, num_gconnections, num_fpconnections, num_fgconnections):
    if num_fpconnections + num_fgconnections < 2:
        return "ucf"
    elif num_pconnections == 1 and num_gconnections ==0:# and num_mconnections==4: #gtotallengths>48
        return "p"
    elif num_pconnections == 0 and num_gconnections ==1:# and num_mconnections == 4:
        return "g"
    elif num_pconnections == 1 and num_gconnections ==1:# and num_mconnections ==3:
        return "b"
    else:
        return "ucf"
    
def connect(cv, radii, bloblist): # center of blob2, radius of all blobs, list of other blobs
    num_connections = 0
    num_f_connections = 0
    norm_radius = 15
    total_separations = 0
    normalized_dists = []
    for blob2 in bloblist:
        cv2 = (blob2[1], blob2[0]) # center of blob2
        vector_d = vector_dist(cv, cv2)
        total_separations += vector_d
        r1,r2 = radii
        if vector_d <= 2*r1 and vector_d > .2*r1: # blob centers closer than diameter of one blob
            num_connections += 1
        if vector_d <= 2*r2 and vector_d > .2*r1: # blob centers closer than diameter of one blob
            num_f_connections += 1
        else:
            pass
        if vector_d <= 4*r1: 
            normalized_dists.append(vector_d/(2*norm_radius))
    return (num_connections, num_f_connections)

def reducer(connections):
    if len(connections)== 0:
        return [0, 0]
    else:
        return(np.sum(connections, axis=0)    )


def vector_dist(v1, v2): # euclidean distance between 2 points
    return math.sqrt(np.sum([(v1[i] - v2[i])**2 for i in range(len(v1))]))

def third_item(l1):
    return l1[2]   

"""Pipeline for lists of frames"""
""""""
"""Pipeline for videos:"""
# initialize frame lists
filenames =glob("./tchakamau/W*half/*_21*a.avi")  #[1,2,3,4,6,7,8,9]["./tchakamau/New_setup/45681_10Vpp_25.avi"
filenames.sort()
print(filenames)
num_top_keys = 4
num_full_keys = 4
num_tran_keys = 3
trans_threshes = np.linspace(25, 65, 4)#[30, 40, 50]#[5, 8, 10, 19, 27, 38, 52, 60]
class_threshes =[20,30,40]
trans_runs = {}
#trans_fil = 10

ave = np.average
expected_blobs = 4
expected_connections = 5
p_bond = 15*1.35
g_bond = 15*1.15
fp_bond = 15*1.75
fg_bond = 15*1.8
    
"""area_threshold = 1800   # must have enough particles
convex_thresholds = (2000, 2800)   # convex hull area (> 2500 filtered)  # particles must be in parallelogram
minor_thresholds = (38, 48) # minor_axis length (< filterd) # particles must be in parallelogram
major_thresholds = (59, 76)"""
thresholds = [expected_blobs , expected_connections, p_bond, g_bond]
params = [3, 115, 98, 2, 95, 80]
print("Number files: ", len(filenames))
index = 0
for trans_fil in trans_threshes:
    
    for class_thresh in class_threshes:
        
        Top_bottoms = np.zeros((len(filenames), num_top_keys))
        Full_classif = np.zeros((len(filenames), num_full_keys))
        N_transitions = np.zeros((len(filenames), num_tran_keys, num_tran_keys))
        num_total_frames = 0 
        filtrate_len = 0
        for vidnum in range(len(filenames)):
            print("Processing vid %s : %s" %(vidnum, filenames[vidnum]))
            #meta = ffprobe(filenames[vidnum])
            #print(meta)#['@nb_frames'])
            #print(5)
            frames = vreader(filenames[vidnum])
            #frame_vid = frames[:, :, :, 2]# making videos ino a frame list
            #print(frames)



            sides = {"p":[], "g":[], "b":[], "ucf":[]}
            classes = {"p":[], "g":[], "b":[], "ucf":[]}
            transitions = [("p->p",[]),("p->g",[]), ("p->b",[]),
                           ("g->p",[]),("g->g",[]), ("g->b",[]),
                          ("b->p",[]),("b->g",[]), ("b->b",[])]

            transitions = OrderedDict(transitions)
            broken_count = 0
            class_num = 0
            origin = 'ucf'
            last_class = 'ucf'
            last_whole = 0
            for fr in frames:
                frame = fr[:, :, 2]
                num_total_frames += 1
                org = total_threshold_filter(frame, num_total_frames, trans_fil, class_thresh,
                                             broken_count, class_num, origin, last_class, last_whole, filtrate_len, 
                                             sides, classes, transitions)
                broken_count, class_num, origin, last_class, last_whole, filtrate_len = org
            filtered_len = filtrate_len
            #----------------------------------------------------------------
            trans_len = len([y for x in transitions.values() for y in x])
            skeys = list(sides.keys())
            ckeys = list(classes.keys())
            tkeys = list(transitions.keys())
            slcs = [len(sides[x]) for x in skeys]
            clcs = [len(classes[x]) for x in ckeys]
            trans = [len(transitions[x]) for x in tkeys] #num of each transition
            ucf = len(sides["ucf"])


            Top_bottoms[vidnum,:] = [slcs[i] for i in range(len(skeys))]
            Full_classif[vidnum,:] = [clcs[i] for i in range(len(ckeys))]
            N_transitions[vidnum,:] = np.reshape(trans, (num_tran_keys, num_tran_keys))
            print("T threshold: %s" %trans_fil)
            print("C threshold: %s" %class_thresh)
            print(N_transitions[vidnum,:])
            #print("finished vid %s" %vidnum)
        trans_runs[index] = [[Top_bottoms, Full_classif, N_transitions, (trans_fil, class_thresh), num_top_keys, num_full_keys,
                                 num_tran_keys, skeys, ckeys, num_tran_keys, num_total_frames], thresholds, params]
        index += 1
        #---------------------------------------------------------------------------------

#-----------------------------------------------------------------------

np.save('trans_runs_15.5.npy',trans_runs)


In [4]:
"""Pipeline for lists of frames"""
""""""
"""Pipeline for videos:"""
# initialize frame lists
filenames =glob("./tchakamau/W*half/*_[4,8]*.avi") +glob("./tchakamau/W*half/*a.avi")#glob("./tchakamau/W*half/*_21*a.avi")  #[1,2,3,4,6,7,8,9]["./tchakamau/New_setup/45681_10Vpp_25.avi"
filenames.sort()
print(filenames)
num_top_keys = 4
num_full_keys = 4
num_tran_keys = 3
num_lengths = 3 # ways to measure the time betwen transitions, as it were
trans_threshes = np.linspace(40, 60, 2)#[30, 40, 50]#[5, 8, 10, 19, 27, 38, 52, 60]
class_threshes =[22,35]
trans_runs = {}
#trans_fil = 10

ave = np.average
expected_blobs = 4
expected_connections = 5
p_bond = 15*1.35
g_bond = 15*1.15
fp_bond = 15*1.75
fg_bond = 15*1.8
    
"""area_threshold = 1800   # must have enough particles
convex_thresholds = (2000, 2800)   # convex hull area (> 2500 filtered)  # particles must be in parallelogram
minor_thresholds = (38, 48) # minor_axis length (< filterd) # particles must be in parallelogram
major_thresholds = (59, 76)"""
thresholds = [expected_blobs , expected_connections, p_bond, g_bond]
params = [3, 115, 98, 2, 95, 80]
print("Number files: ", len(filenames))
index = 0
for trans_fil in trans_threshes:
    
    for class_thresh in class_threshes:
        
        #Top_bottoms = np.zeros((len(filenames), num_top_keys))
        Full_classif = np.zeros((len(filenames), num_full_keys))
        N_transitions = np.zeros((len(filenames), num_tran_keys, num_tran_keys))
        T_transitions = [[[[[] for i in range(num_tran_keys)] for j in range(num_tran_keys)] for k in range(num_lengths)] 
                          for l in range(len(filenames))]
        num_total_frames = 0 
        filtrate_len = 0
        for vidnum in range(len(filenames)):
            print("Processing vid %s : %s" %(vidnum, filenames[vidnum]))
            #meta = ffprobe(filenames[vidnum])
            #print(meta)#['@nb_frames'])
            #print(5)
            frames = vreader(filenames[vidnum])
            #frame_vid = frames[:, :, :, 2]# making videos ino a frame list
            #print(frames)



            
            classes = [("p",0), ("g",0), ("b",0), ("ucf",0)]
            transitions = [("p->p",0),("p->g",0), ("p->b",0),
                           ("g->p",0),("g->g",0), ("g->b",0),
                          ("b->p",0),("b->g",0), ("b->b",0)]
            times = [("p->p",[[] for i in range(num_lengths)]),("p->g",[[] for i in range(num_lengths)]), ("p->b",[[] for i in range(num_lengths)]),
                           ("g->p",[[] for i in range(num_lengths)]),("g->g",[[] for i in range(num_lengths)]), ("g->b",[[] for i in range(num_lengths)]),
                          ("b->p",[[] for i in range(num_lengths)]),("b->g",[[] for i in range(num_lengths)]), ("b->b",[[] for i in range(num_lengths)])]

            transitions = OrderedDict(transitions)
            times = OrderedDict(times)            
            classes = OrderedDict(classes)
            broken_count = 0
            class_num = 0
            origin = 'ucf'
            last_class = 'ucf'
            last_whole = 0
            state_start = 0
            frame_no = 0
            for fr in frames:
                frame = fr[:, :, 2]
                frame_no += 1
                num_total_frames += 1
                org = total_threshold_filter(frame, frame_no, trans_fil, class_thresh,
                                             broken_count, class_num, origin, last_class, last_whole, state_start, filtrate_len, 
                                              classes, transitions, times)
                broken_count, class_num, origin, last_class, last_whole, state_start, filtrate_len = org
            filtered_len = filtrate_len
            #----------------------------------------------------------------
            #trans_len = len([y for x in transitions.values() for y in x])
            
            ckeys = list(classes.keys())
            clcs = [classes[x] for x in ckeys] # list of frame nums per class
            
            tkeys = list(times.keys())
            tims = [times[x] for x in tkeys]
            
            trkeys = list(transitions.keys())
            trans = [transitions[x] for x in trkeys] #num of each transition
            #ucf = len(sides["ucf"])

            #Top_bottoms[vidnum,:] = [slcs[i] for i in range(len(skeys))]
            Full_classif[vidnum,:] = clcs
            N_transitions[vidnum,:] = np.reshape(trans, (num_tran_keys, num_tran_keys))            
            T_transitions[vidnum] = np.moveaxis(np.reshape(tims, (num_tran_keys, num_tran_keys,num_lengths)),
                                                  [0,1,2], [1,2,0])
            print("T threshold: %s" %trans_fil)
            print("C threshold: %s" %class_thresh)
            print(N_transitions[vidnum,:])
            print(T_transitions[vidnum])
            print(Full_classif[vidnum,:])
            #print("finished vid %s" %vidnum)
        trans_runs[index] = [[Full_classif, (N_transitions, T_transitions), (trans_fil, class_thresh), num_top_keys, num_full_keys,
                                 num_tran_keys, ckeys, num_tran_keys, num_total_frames], thresholds, params]
        index += 1
        #---------------------------------------------------------------

#-----------------------------------------------------------------------

np.save('trans_runs_15.5_timesbetween.npy',trans_runs)


['./tchakamau\\Washed15vhalf\\45657_15Vpp_101_a.avi', './tchakamau\\Washed15vhalf\\45662_15Vpp_111_a.avi', './tchakamau\\Washed15vhalf\\45686_15Vpp_131_pushed_a.avi', './tchakamau\\Washed15vhalf\\45693_15Vpp_121_a.avi', './tchakamau\\Washed15vhalf\\45706_15Vpp_71_a.avi', './tchakamau\\Washed15vhalf\\45706_15Vpp_72_ba.avi', './tchakamau\\Washed15vhalf\\45734_15Vpp_8.avi', './tchakamau\\Washed15vhalf\\45755_15Vpp_141_a.avi', './tchakamau\\Washed15vhalf\\45782_15Vpp_21_a.avi', './tchakamau\\Washed15vhalf\\45782_15Vpp_22_ba.avi', './tchakamau\\Washed15vhalf\\45782_15Vpp_31_a.avi', './tchakamau\\Washed15vhalf\\45782_15Vpp_32_ba.avi', './tchakamau\\Washed15vhalf\\45782_15Vpp_33_ca.avi', './tchakamau\\Washed15vhalf\\45782_15Vpp_4.avi']
Number files:  14
Processing vid 0 : ./tchakamau\Washed15vhalf\45657_15Vpp_101_a.avi
T threshold: 40.0
C threshold: 22
[[0. 0. 1.]
 [0. 0. 0.]
 [0. 0. 6.]]
[[[list([]) list([]) list([3757])]
  [list([]) list([]) list([])]
  [list([]) list([]) list([1907, 445, 2

T threshold: 40.0
C threshold: 22
[[2. 0. 1.]
 [0. 0. 0.]
 [0. 0. 4.]]
[[[list([459, 406]) list([]) list([2350])]
  [list([]) list([]) list([])]
  [list([]) list([]) list([883, 356, 812, 248])]]

 [[list([0, 241]) list([]) list([544])]
  [list([]) list([]) list([])]
  [list([]) list([]) list([773, 122, 723, 162])]]

 [[list([459, 165]) list([]) list([1806])]
  [list([]) list([]) list([])]
  [list([]) list([]) list([110, 234, 89, 86])]]]
[8.630e+02 2.000e+00 2.146e+03 2.697e+03]
Processing vid 12 : ./tchakamau\Washed15vhalf\45782_15Vpp_33_ca.avi
T threshold: 40.0
C threshold: 22
[[3. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
[[[list([3541, 1905, 641]) list([]) list([])]
  [list([]) list([]) list([])]
  [list([]) list([]) list([])]]

 [[list([1966, 1750, 392]) list([]) list([])]
  [list([]) list([]) list([])]
  [list([]) list([]) list([])]]

 [[list([1575, 155, 249]) list([]) list([])]
  [list([]) list([]) list([])]
  [list([]) list([]) list([])]]]
[5.145e+03 1.000e+00 9.000e+01 4.838e+03]
Process

T threshold: 40.0
C threshold: 35
[[0. 0. 1.]
 [0. 0. 0.]
 [1. 0. 3.]]
[[[list([]) list([]) list([1138])]
  [list([]) list([]) list([])]
  [list([626]) list([]) list([1052, 443, 549])]]

 [[list([]) list([]) list([29])]
  [list([]) list([]) list([])]
  [list([4]) list([]) list([0, 5, 56])]]

 [[list([]) list([]) list([1109])]
  [list([]) list([]) list([])]
  [list([622]) list([]) list([1052, 438, 493])]]]
[ 117.   21. 2011. 1880.]
Processing vid 11 : ./tchakamau\Washed15vhalf\45782_15Vpp_32_ba.avi
T threshold: 40.0
C threshold: 35
[[2. 0. 1.]
 [0. 0. 0.]
 [0. 0. 4.]]
[[[list([472, 406]) list([]) list([2350])]
  [list([]) list([]) list([])]
  [list([]) list([]) list([909, 363, 779, 292])]]

 [[list([0, 228]) list([]) list([531])]
  [list([]) list([]) list([])]
  [list([]) list([]) list([760, 51, 677, 149])]]

 [[list([472, 178]) list([]) list([1819])]
  [list([]) list([]) list([])]
  [list([]) list([]) list([149, 312, 102, 143])]]]
[8.630e+02 2.000e+00 2.146e+03 2.697e+03]
Processing vi

T threshold: 60.0
C threshold: 22
[[0. 0. 1.]
 [0. 0. 0.]
 [1. 0. 4.]]
[[[list([]) list([]) list([954])]
  [list([]) list([]) list([])]
  [list([626]) list([]) list([935, 547, 184, 549])]]

 [[list([]) list([]) list([42])]
  [list([]) list([]) list([])]
  [list([17]) list([]) list([318, 214, 80, 124])]]

 [[list([]) list([]) list([912])]
  [list([]) list([]) list([])]
  [list([609]) list([]) list([617, 333, 104, 425])]]]
[ 117.   21. 2011. 1880.]
Processing vid 11 : ./tchakamau\Washed15vhalf\45782_15Vpp_32_ba.avi
T threshold: 60.0
C threshold: 22
[[2. 0. 1.]
 [0. 0. 0.]
 [0. 0. 2.]]
[[[list([459, 406]) list([]) list([2350])]
  [list([]) list([]) list([])]
  [list([]) list([]) list([1239, 1060])]]

 [[list([0, 241]) list([]) list([544])]
  [list([]) list([]) list([])]
  [list([]) list([]) list([1005, 974])]]

 [[list([459, 165]) list([]) list([1806])]
  [list([]) list([]) list([])]
  [list([]) list([]) list([234, 86])]]]
[8.630e+02 2.000e+00 2.146e+03 2.697e+03]
Processing vid 12 : ./tc

T threshold: 60.0
C threshold: 35
[[0. 0. 1.]
 [0. 0. 0.]
 [1. 0. 3.]]
[[[list([]) list([]) list([1138])]
  [list([]) list([]) list([])]
  [list([626]) list([]) list([1052, 443, 549])]]

 [[list([]) list([]) list([29])]
  [list([]) list([]) list([])]
  [list([4]) list([]) list([0, 5, 56])]]

 [[list([]) list([]) list([1109])]
  [list([]) list([]) list([])]
  [list([622]) list([]) list([1052, 438, 493])]]]
[ 117.   21. 2011. 1880.]
Processing vid 11 : ./tchakamau\Washed15vhalf\45782_15Vpp_32_ba.avi
T threshold: 60.0
C threshold: 35
[[2. 0. 1.]
 [0. 0. 0.]
 [0. 0. 2.]]
[[[list([472, 406]) list([]) list([2350])]
  [list([]) list([]) list([])]
  [list([]) list([]) list([1272, 1071])]]

 [[list([0, 228]) list([]) list([531])]
  [list([]) list([]) list([])]
  [list([]) list([]) list([960, 928])]]

 [[list([472, 178]) list([]) list([1819])]
  [list([]) list([]) list([])]
  [list([]) list([]) list([312, 143])]]]
[8.630e+02 2.000e+00 2.146e+03 2.697e+03]
Processing vid 12 : ./tchakamau\Washed15

In [5]:
print(np.moveaxis(np.reshape(tims, (num_tran_keys, num_tran_keys,num_lengths)),
                                                  [0,1,2], [1,2,0]))

print(vidnum)

T_transitions = [[[[[] for i in range(num_tran_keys)] for j in range(num_tran_keys)] for k in range(num_lengths)] 
                          for l in range(len(filenames))]
#print(T_transitions)
T_transitions[vidnum] = np.moveaxis(np.reshape(tims, (num_tran_keys, num_tran_keys,num_lengths)),
                                                  [0,1,2], [1,2,0])
print(T_transitions, len(T_transitions), vidnum)

[[[list([714, 1483, 498, 528, 1597]) list([]) list([805, 1269, 818])]
  [list([]) list([]) list([])]
  [list([1693, 1366, 620]) list([]) list([354])]]

 [[list([576, 1124, 0, 18, 19]) list([]) list([0, 5, 110])]
  [list([]) list([]) list([])]
  [list([1, 275, 364]) list([]) list([222])]]

 [[list([138, 359, 498, 510, 1578]) list([]) list([805, 1264, 708])]
  [list([]) list([]) list([])]
  [list([1692, 1091, 256]) list([]) list([132])]]]
13
[[[[[], [], []], [[], [], []], [[], [], []]], [[[], [], []], [[], [], []], [[], [], []]], [[[], [], []], [[], [], []], [[], [], []]]], [[[[], [], []], [[], [], []], [[], [], []]], [[[], [], []], [[], [], []], [[], [], []]], [[[], [], []], [[], [], []], [[], [], []]]], [[[[], [], []], [[], [], []], [[], [], []]], [[[], [], []], [[], [], []], [[], [], []]], [[[], [], []], [[], [], []], [[], [], []]]], [[[[], [], []], [[], [], []], [[], [], []]], [[[], [], []], [[], [], []], [[], [], []]], [[[], [], []], [[], [], []], [[], [], []]]], [[[[], [], []], [[]